In [1]:
%reload_ext autoreload
%autoreload 2
%matplotlib inline

In [2]:
from imports import *
from torch_imports import *
from torchtext import vocab, data, datasets

In [3]:
from torch.nn.init import *

In [4]:
from pt_models import *
from dataset_pt import *

from sgdr_pt import *

### IMDB test

In [5]:
class TextDataLoader():
    def __init__(self, src, x_fld, y_fld):
        self.src,self.x_fld,self.y_fld = src,x_fld,y_fld
        
    def __len__(self): return len(self.src)
    
    def __iter__(self): 
        try:
            for b in iter(self.src):
                yield getattr(b, self.x_fld), getattr(b, self.y_fld).float()
        except StopIteration: return

def create_td(it): return TextDataLoader(it, 'text', 'label')

In [87]:
def postproc(arr,vocab,train): 
    return [1 if o==1 else 0 for o in arr]

In [88]:
# set up fields
TEXT = data.Field(lower=True, fix_length=500)
LABEL = data.Field(sequential=False, postprocessing=postproc)

# make splits for data
trn, test = datasets.IMDB.splits(TEXT, LABEL, root='tmp/')

In [89]:
# print information about the data
print('trn.fields', trn.fields)
print('len(trn)', len(trn))
print('vars(trn[0])', vars(trn[0]))

trn.fields {'text': <torchtext.data.field.Field object at 0x7fb91016aef0>, 'label': <torchtext.data.field.Field object at 0x7fb91016ae10>}
len(trn) 25000
vars(trn[0]) {'text': ['i', 'remember', 'when', 'i', 'first', 'saw', 'this', 'short,', 'i', 'was', 'really', 'laughing', 'so', 'hard,', 'that', 'like', 'with', 'a', 'lot', 'of', 'other', 'films', 'that', 'i', 'have', 'seen,', 'no', 'sound', 'came', 'out!', 'curly', 'is', 'really', 'great', 'at', '"singing"', 'opera', 'in', 'this', 'one,', 'i', 'am', 'surprised', 'that', 'he', 'did', 'not', 'consider', 'a', 'career', 'as', 'a', 'professional', 'singer,', 'because', 'he', 'was', 'really', 'good!', '<br', '/><br', '/>if', 'you', 'noticed,', 'this', 'was', 'filmed', 'near', 'the', 'end', 'of', "curly's", 'career', 'as', 'a', 'stooge,', 'you', 'could', 'really', 'tell', 'he', 'had', 'changed,', 'because', 'he', 'had', 'lost', 'weight', 'and', 'was', 'thinner,', 'his', 'voice', 'was', 'deepening,', 'his', 'face', 'was', 'getting', 'lined', 

In [90]:
vocab_size=5000

In [91]:
# build the vocabulary
TEXT.build_vocab(trn, vectors='glove.6B.50d', max_size=vocab_size)
LABEL.build_vocab(trn)

# print vocab information
print('len(TEXT.vocab)', len(TEXT.vocab))
print('TEXT.vocab.vectors.size()', TEXT.vocab.vectors.size())

loading vectors from .vector_cache/glove.6B.50d.pt
len(TEXT.vocab) 5002
TEXT.vocab.vectors.size() torch.Size([5002, 50])


In [92]:
LABEL.vocab.itos

['<unk>', 'neg', 'pos']

In [93]:
TEXT.vocab.vectors


 0.0000  0.0000  0.0000  ...   0.0000  0.0000  0.0000
 0.0000  0.0000  0.0000  ...   0.0000  0.0000  0.0000
 0.4180  0.2497 -0.4124  ...  -0.1841 -0.1151 -0.7858
          ...             ⋱             ...          
 0.0000  0.0000  0.0000  ...   0.0000  0.0000  0.0000
-1.3515  0.5061 -1.9024  ...  -0.3654 -0.6569 -0.7150
 0.0000  0.0000  0.0000  ...   0.0000  0.0000  0.0000
[torch.FloatTensor of size 5002x50]

In [94]:
trn_iter, test_iter = data.BucketIterator.splits((trn, test), batch_size=64, repeat=False)

In [95]:
trn_iter, test_iter = data.Iterator.splits((trn, test), batch_size=64)

In [96]:
[len(trn[i].text) for i in range(20)]

[199,
 118,
 515,
 128,
 194,
 155,
 198,
 135,
 127,
 129,
 148,
 157,
 57,
 367,
 528,
 78,
 229,
 161,
 117,
 166]

In [97]:
batch = next(iter(trn_iter))

In [98]:
print(batch.text[-300:,:10])

Variable containing:
    1     0     1  ...      0   319     1
    1   310     1  ...   3052     0     1
    1    37     1  ...    729  1071     1
       ...          ⋱          ...       
    1     1     1  ...      1     1     1
    1     1     1  ...      1     1     1
    1     1     1  ...      1     1     1
[torch.cuda.LongTensor of size 300x10 (GPU 0)]



In [99]:
print(batch.text[-300:,:10])

Variable containing:
    1     0     1  ...      0   319     1
    1   310     1  ...   3052     0     1
    1    37     1  ...    729  1071     1
       ...          ⋱          ...       
    1     1     1  ...      1     1     1
    1     1     1  ...      1     1     1
    1     1     1  ...      1     1     1
[torch.cuda.LongTensor of size 300x10 (GPU 0)]



In [100]:
print(batch.label)

Variable containing:
 1
 0
 1
 0
 0
 1
 1
 0
 1
 1
 0
 0
 0
 0
 1
 0
 0
 1
 1
 1
 1
 1
 0
 0
 1
 1
 0
 1
 0
 0
 1
 1
 0
 1
 1
 1
 0
 1
 1
 1
 0
 1
 1
 0
 1
 1
 0
 0
 0
 1
 1
 1
 0
 1
 0
 0
 0
 1
 1
 1
 0
 1
 0
 1
[torch.cuda.LongTensor of size 64 (GPU 0)]



In [101]:
TEXT.vocab.vectors[:10,:10]


 0.0000  0.0000  0.0000  0.0000  0.0000  0.0000  0.0000  0.0000  0.0000  0.0000
 0.0000  0.0000  0.0000  0.0000  0.0000  0.0000  0.0000  0.0000  0.0000  0.0000
 0.4180  0.2497 -0.4124  0.1217  0.3453 -0.0445 -0.4969 -0.1786 -0.0007 -0.6566
 0.2171  0.4651 -0.4676  0.1008  1.0135  0.7484 -0.5310 -0.2626  0.1681  0.1318
 0.2682  0.1435 -0.2788  0.0163  0.1138  0.6992 -0.5133 -0.4737 -0.3307 -0.1383
 0.7085  0.5709 -0.4716  0.1805  0.5445  0.7260  0.1816 -0.5239  0.1038 -0.1757
 0.6805 -0.0393  0.3019 -0.1779  0.4296  0.0322 -0.4138  0.1323 -0.2985 -0.0853
 0.6185  0.6425 -0.4655  0.3757  0.7484  0.5374  0.0022 -0.6058  0.2641  0.1170
 0.3304  0.2500 -0.6087  0.1092  0.0364  0.1510 -0.5508 -0.0742 -0.0923 -0.3282
 0.1189  0.1525 -0.0821 -0.7414  0.7592 -0.4833 -0.3101  0.5148 -0.9871  0.0006
[torch.FloatTensor of size 10x10]

In [102]:
len(TEXT.vocab.freqs), len(TEXT.vocab.itos)

(251637, 5002)

In [103]:
pad=TEXT.vocab.stoi['<pad>']

In [104]:
md = ModelData('.', create_td(trn_iter), create_td(test_iter))

In [230]:
nf=100
n_emb=32
bs=64
nv=len(TEXT.vocab.itos)

In [783]:
def init_wgts(m, last_l=-2):
    c = children(m)
    for l in c:
        if isinstance(l, nn.Embedding): 
            l.weight.data.uniform_(-0.05,0.05)
        elif isinstance(l, (nn.Linear, nn.Conv1d)):
            xavier_uniform(l.weight.data, gain=calculate_gain('relu'))
            l.bias.data.zero_()
    xavier_uniform(c[last_l].weight.data, gain=calculate_gain('linear'));

In [232]:
m = nn.Sequential(
    nn.Embedding(nv, n_emb, padding_idx=pad, sparse=True),
    nn.Dropout(0.01),
    Lambda(lambda x: x.transpose(0,1).contiguous().view(x.size(1),-1)),
    nn.Linear(500*n_emb, nf),
    nn.ReLU(),
    nn.Dropout(0.7),
    nn.Linear(nf, 1),
    Lambda(lambda x: x.squeeze())
).cuda()

init_wgts(m)

In [233]:
opt = optim.Adagrad(trainable_params_(m), 1e-2)

In [234]:
fit(m, md, 2, F.binary_cross_entropy_with_logits, opt, metrics=[accuracy_thresh(0.5)])

[ 0.416624  0.402875  0.836205]


/home/jhoward/anaconda3/lib/python3.6/site-packages/ipykernel/__main__.py:9: DeprecationWarning: generator 'Iterator.__iter__' raised StopIteration


[ 0.262659  0.334411  0.854348]


In [215]:
class SeqSize(nn.Sequential):
    def forward(self, x):
        for l in self.children():
            x = l(x)
            print(x.size())
        return x

In [261]:
??F.dropout2d

In [289]:
m = nn.Sequential(
    nn.Embedding(nv, n_emb, padding_idx=pad, sparse=True),
    Lambda(lambda x: x.permute(1,2,0).contiguous()),
    nn.Conv1d(n_emb, n_emb, 5, padding=2, stride=2),
    nn.ReLU(),
    nn.Dropout2d(0.25),
    nn.Conv1d(n_emb, n_emb, 5, padding=2, stride=2),
    nn.ReLU(),
    nn.Dropout2d(0.5),
    Flatten(),
    nn.Linear(n_emb*500//4, 100),
    nn.ReLU(),
    nn.Dropout(0.5),
    nn.Linear(100, 1),
    Lambda(lambda x: x.squeeze())
).cuda()

init_wgts(m)

In [290]:
opt = optim.Adagrad(trainable_params_(m), 1e-2)

In [291]:
set_lrs(opt,1e-2)

In [292]:
fit(m, md, 3, F.binary_cross_entropy_with_logits, opt, metrics=[accuracy_thresh(0.5)])

[ 0.675121  0.635704  0.530555]


/home/jhoward/anaconda3/lib/python3.6/site-packages/ipykernel/__main__.py:9: DeprecationWarning: generator 'Iterator.__iter__' raised StopIteration


[ 0.411029  0.342623  0.823066]


[ 0.337649  0.303579  0.863451]


In [293]:
set_lrs(opt,1e-3)

In [294]:
fit(m, md, 2, F.binary_cross_entropy_with_logits, opt, metrics=[accuracy_thresh(0.5)])

[ 0.294003  0.299945  0.861493]


/home/jhoward/anaconda3/lib/python3.6/site-packages/ipykernel/__main__.py:9: DeprecationWarning: generator 'Iterator.__iter__' raised StopIteration


[ 0.293427  0.298573  0.862972]


### LSTM

In [296]:
def postproc(arr,vocab,train): 
    return [1 if o==1 else 0 for o in arr]

In [297]:
TEXT = data.Field(lower=True)
LABEL = data.Field(sequential=False, postprocessing=postproc)
trn, test = datasets.IMDB.splits(TEXT, LABEL, root='tmp/')

In [298]:
vocab_size=5000

In [509]:
def init_emb(x): return x.uniform_(-0.05,0.05)

In [510]:
TEXT.build_vocab(trn, vectors='glove.6B.50d', max_size=vocab_size, unk_init=init_emb)
LABEL.build_vocab(trn)

loading vectors from .vector_cache/glove.6B.50d.pt


In [511]:
pad=TEXT.vocab.stoi['<pad>']

In [512]:
trn_iter, test_iter = data.BucketIterator.splits((trn, test), batch_size=64, repeat=False)

In [513]:
md = ModelData('.', create_td(trn_iter), create_td(test_iter))

In [514]:
def init_wgts(m):
    c = children(m)
    for l in c:
        if isinstance(l, nn.Embedding): 
            l.weight.data.uniform_(-0.05,0.05)
        elif isinstance(l, (nn.Linear, nn.Conv1d)):
            xavier_uniform(l.weight.data, gain=calculate_gain('relu'))
            l.bias.data.zero_()
    xavier_uniform(c[-2].weight.data, gain=calculate_gain('linear'));

In [515]:
class EzLSTM(nn.LSTM):
    def __init__(self, input_size, hidden_size, *args, **kwargs):
        super().__init__(input_size, hidden_size, *args, **kwargs)
        self.num_dirs = 2 if self.bidirectional else 1
        self.input_size = input_size
        self.hidden_size = hidden_size
        
    def forward(self, x):
        h0 = c0 = Variable(torch.zeros(self.num_dirs,x.size(1),self.hidden_size)).cuda()
        outp,_ = super().forward(x, (h0,c0))
        return outp[-1]

In [516]:
nv=len(TEXT.vocab.itos)
hs=70
n_emb=50

In [610]:
m = nn.Sequential(
    nn.Embedding(nv, n_emb, padding_idx=pad, sparse=True),
    nn.Dropout(0.4),
    EzLSTM(n_emb, hs, num_layers=1, bidirectional=False),
    nn.Dropout(0.2),
    nn.Linear(hs*1, 1),
    Lambda(lambda x: x.squeeze())
).cuda()

init_wgts(m)

m[0].weight.data = TEXT.vocab.vectors.cuda()

In [611]:
def get_opt(p): return optim.Adam(p, betas=(0.,0.999))

cl=4
lrs=np.array([1e-2, 1e-2])
nb=len(md.trn_dl)
t=list(split_by_idxs(list(m.children()), [1]))
# lo = LayerOptimizer(optim.Adagrad, t, lrs)
lo = LayerOptimizer(get_opt, t, lrs)
cb=[CosAnneal(lo, nb*cl)]

In [612]:
opt = lo.opt

In [614]:
fit(m, md, 4, F.binary_cross_entropy_with_logits, opt, 
    metrics=[accuracy_thresh(0.5)], callbacks=cb)

[ 0.334644  0.330858  0.865769]


/home/jhoward/anaconda3/lib/python3.6/site-packages/ipykernel/__main__.py:9: DeprecationWarning: generator 'Iterator.__iter__' raised StopIteration


[ 0.27686   0.305416  0.872882]


[ 0.264897  0.308693  0.873777]


[ 0.248931  0.306658  0.874137]


### CNN/RNN

In [758]:
m = nn.Sequential(
    nn.Embedding(nv, n_emb, padding_idx=pad),
    nn.Dropout(0.4),
    Lambda(lambda x: x.permute(1,2,0).contiguous()), # b/c/s
    nn.Conv1d(n_emb, n_emb*2, 7, padding=3, stride=5),
    nn.ReLU(),
    nn.Dropout2d(0.4),
    Lambda(lambda x: x.permute(2,0,1).contiguous()),   # s/b/c
    EzLSTM(n_emb*2, hs, num_layers=1, bidirectional=False),
    nn.Dropout(0.1),
    nn.Linear(hs*1, 1),
    Lambda(lambda x: x.squeeze())
).cuda()

init_wgts(m)
m[0].weight.data = TEXT.vocab.vectors.cuda()

In [759]:
def get_opt(p): return optim.Adam(p, betas=(0.,0.999))

cl=4
lrs=np.array([1e-2, 1e-2])/2
nb=len(md.trn_dl)
t=list(split_by_idxs(list(m.children()), [1]))
lo = LayerOptimizer(get_opt, t, lrs)
cb=[CosAnneal(lo, nb*cl)]

In [760]:
opt = lo.opt

In [761]:
fit(m, md, 4, F.binary_cross_entropy_with_logits, opt, 
    metrics=[accuracy_thresh(0.5)], callbacks=cb)

[ 0.543035  0.444902  0.804092]


/home/jhoward/anaconda3/lib/python3.6/site-packages/ipykernel/__main__.py:9: DeprecationWarning: generator 'Iterator.__iter__' raised StopIteration


[ 0.373775  0.329732  0.828341]


[ 0.305102  0.297006  0.86927 ]


[ 0.281998  0.296998  0.867983]


In [762]:
fit(m, md, 4, F.binary_cross_entropy_with_logits, opt, 
    metrics=[accuracy_thresh(0.5)], callbacks=cb)

[ 0.295826  0.286968  0.881977]


/home/jhoward/anaconda3/lib/python3.6/site-packages/ipykernel/__main__.py:9: DeprecationWarning: generator 'Iterator.__iter__' raised StopIteration


[ 0.255057  0.29808   0.864154]


[ 0.217908  0.296243  0.875967]


[ 0.223294  0.294551  0.882041]


### StatsModel

In [858]:
class StatsModel(nn.Module):
    def __init__(self):
        super().__init__()
        self.emb = nn.Embedding(nv, n_emb)
        self.l1 = nn.Linear(n_emb*2,10)
        self.drop1 = nn.Dropout(0.5)
        self.drop2 = nn.Dropout(0.5)
        self.l2 = nn.Linear(50,1)
        
    def forward(self, x):
        x = self.emb(x)
        x = torch.cat([x.mean(0), x.max(0)], 0)
        x = self.drop1(x)
        x = F.relu(self.l1(x))
        x = nn.BatchNorm1d(x)
        x = self.drop2(x)
        x = self.l2(x)
        return x.squeeze()
        
init_wgts(m, -1)
m.emb.weight.data = TEXT.vocab.vectors.cuda()

In [859]:
m = SimpleAverage().cuda()

In [863]:
def get_opt(p): return optim.Adam(p, betas=(0.,0.999))

cl=4
lrs=1e-2
nb=len(md.trn_dl)
t=list(split_by_idxs(list(m.children()), [1]))
lo = LayerOptimizer(get_opt, t, lrs)
cb=[CosAnneal(lo, nb*cl)]

In [864]:
opt = lo.opt

In [865]:
fit(m, md, 4, F.binary_cross_entropy_with_logits, opt, 
    metrics=[accuracy_thresh(0.5)], callbacks=cb)

[ 0.274308  0.32876   0.865473]


/home/jhoward/anaconda3/lib/python3.6/site-packages/ipykernel/__main__.py:9: DeprecationWarning: generator 'Iterator.__iter__' raised StopIteration


[ 0.244463  0.331872  0.866152]


[ 0.194321  0.354025  0.86469 ]


[ 0.160523  0.370321  0.865353]


### COVE

In [6]:
from torch.nn.utils.rnn import pad_packed_sequence as unpack
from torch.nn.utils.rnn import pack_padded_sequence as pack
import torch.utils.model_zoo as model_zoo

In [7]:
def postproc(arr,vocab,train): 
    return [1 if o==1 else 0 for o in arr]

In [8]:
TEXT = data.Field(lower=True)
LABEL = data.Field(sequential=False, postprocessing=postproc)
trn, test = datasets.IMDB.splits(TEXT, LABEL, root='tmp/')

In [9]:
def init_emb(x): return x.uniform_(-0.05,0.05)

In [10]:
vocab_size=5000

TEXT.build_vocab(trn, vectors='glove.6B.300d', max_size=vocab_size, unk_init=init_emb)
LABEL.build_vocab(trn)

loading vectors from .vector_cache/glove.6B.300d.pt


In [11]:
nv=len(TEXT.vocab.itos)
hs=70
n_emb=50

In [12]:
pad=TEXT.vocab.stoi['<pad>']
trn_iter, test_iter = data.BucketIterator.splits((trn, test), batch_size=64, repeat=False)
md = ModelData('.', create_td(trn_iter), create_td(test_iter))

In [13]:
model_urls = {
    'wmt-lstm' : 'https://s3.amazonaws.com/research.metamind.io/cove/wmtlstm-b142a7f2.pth'
}

model_cache = os.path.join('tmp/', '.torch')

In [14]:
class MTLSTM(nn.Module):
    def __init__(self):
        super().__init__()
        self.emb = nn.Embedding(nv, n_emb)
        self.rnn = nn.LSTM(300, 300, num_layers=2, bidirectional=True)
        self.rnn.load_state_dict(model_zoo.load_url(model_urls['wmt-lstm'], model_dir=model_cache))
        self.drop = nn.Dropout(0.75)
        self.l1 = nn.Linear(600, 1)

    def forward(self, x):
        x = self.emb(x)
        x, _ = self.rnn(x, None)
        x = self.l1(self.drop(x[-1]))
        return x.squeeze()

m = MTLSTM().cuda()
m.emb.weight.data = TEXT.vocab.vectors.cuda()
for p in m.rnn.parameters(): p.requires_grad=False

In [15]:
def get_opt(p): return optim.Adam(p, betas=(0.,0.999))

In [16]:
cl=1
lrs=1e-2*2
nb=len(md.trn_dl)
t=list(split_by_idxs(list(m.children()), [1]))

In [17]:
lo = LayerOptimizer(get_opt, t, lrs)
cb=[CosAnneal(lo, nb*cl)]
opt = lo.opt

In [18]:
fit(m, md, 1, F.binary_cross_entropy_with_logits, opt, 
    metrics=[accuracy_thresh(0.5)], callbacks=cb)

[ 0.497202  0.485905  0.760758]



/home/jhoward/anaconda3/lib/python3.6/site-packages/ipykernel/__main__.py:9: DeprecationWarning: generator 'Iterator.__iter__' raised StopIteration


In [19]:
for p in m.emb.parameters(): p.requires_grad=False

In [20]:
set_lrs(opt,lrs/2)

In [21]:
fit(m, md, 1, F.binary_cross_entropy_with_logits, opt, 
    metrics=[accuracy_thresh(0.5)], callbacks=cb)

[ 0.379714  0.474497  0.776455]



/home/jhoward/anaconda3/lib/python3.6/site-packages/ipykernel/__main__.py:9: DeprecationWarning: generator 'Iterator.__iter__' raised StopIteration


In [22]:
for p in m.emb.parameters(): p.requires_grad=True

In [24]:
set_lrs(opt,lrs/2)

In [25]:
fit(m, md, 1, F.binary_cross_entropy_with_logits, opt, 
    metrics=[accuracy_thresh(0.5)], callbacks=cb)

[ 0.484556  0.48486   0.761189]



/home/jhoward/anaconda3/lib/python3.6/site-packages/ipykernel/__main__.py:9: DeprecationWarning: generator 'Iterator.__iter__' raised StopIteration


In [26]:
for p in m.rnn.parameters(): p.requires_grad=True

In [27]:
lo = LayerOptimizer(get_opt, t, lrs/10)
cb=[CosAnneal(lo, nb*cl)]
opt = lo.opt

In [29]:
fit(m, md, 1, F.binary_cross_entropy_with_logits, opt, 
    metrics=[accuracy_thresh(0.5)], callbacks=cb)

KeyboardInterrupt: 

### Translation test

In [52]:
import re, spacy

In [53]:
spacy_de = spacy.load('de')
spacy_en = spacy.load('en')



    Only loading the 'de' tokenizer.



    Only loading the 'en' tokenizer.



In [54]:
url = re.compile('(<url>.*</url>)')

def tokenize_de(text):
    return [tok.text for tok in spacy_de.tokenizer(url.sub('@URL@', text))]

def tokenize_en(text):
    return [tok.text for tok in spacy_en.tokenizer(url.sub('@URL@', text))]

DE = data.Field(tokenize=tokenize_de)
EN = data.Field(tokenize=tokenize_en)

From https://wit3.fbk.eu/archive/2016-01//texts/de/en/de-en.tgz

In [56]:
path='/data/jhoward/github/deeplearning/nbs/'

In [58]:
trn, val = datasets.TranslationDataset.splits(
    path=path+'tmp/de-en/', train='train.tags.de-en',
    validation='IWSLT16.TED.tst2013.de-en', exts=('.de', '.en'),
    fields=(DE, EN))

print(trn.fields)
print(len(trn))
print(vars(trn[0]))
print(vars(trn[100]))

{'src': <torchtext.data.field.Field object at 0x7f7260609048>, 'trg': <torchtext.data.field.Field object at 0x7f7260609128>}
209772
{'src': ['@URL@'], 'trg': ['@URL@']}
{'src': ['BL', ':', 'Diese', 'Würmer', 'haben', 'kein', 'Verdauungssystem', '.', 'Sie', 'haben', 'keinen', 'Mund', '.'], 'trg': ['BL', ':', 'These', 'worms', 'have', 'no', 'digestive', 'system', '.', 'They', 'have', 'no', 'mouth', '.']}


In [59]:
DE.build_vocab(train.src, min_freq=3)
EN.build_vocab(train.trg, max_size=50000)

In [61]:
train_iter, val_iter = data.BucketIterator.splits(
    (train, val), batch_size=3, device=0)

print(DE.vocab.freqs.most_common(10))
print(len(DE.vocab.freqs))
print(EN.vocab.freqs.most_common(10))
print(len(EN.vocab.freqs))

[(',', 293485), ('.', 205869), ('die', 85250), ('und', 77141), ('der', 56257), ('ist', 51461), ('das', 45803), ('zu', 44547), ('in', 44314), ('ich', 39040)]
136046
[(',', 248240), ('.', 196796), ('the', 155955), ('to', 97498), ('of', 91549), ('and', 84765), ('a', 82235), ('that', 69796), ('I', 63840), ('in', 58086)]
65325


In [64]:
batch = next(iter(train_iter))
print(batch.src)
print(batch.trg)

Variable containing:
    22   1004    104
    24      6     38
    84    754     81
   262   4097    250
     2     95     41
    19   1610    459
  7448     47    202
  1387   3758  24275
  2424   1422  10394
    17      7    816
     0      2      2
  1070     19      8
     7    827     66
     2   1001     11
   737      2     94
    11   1117    124
    44   2107      2
  6154      5    157
     2     11     59
   145    221      7
    12     63    166
    15  13079   4817
  1164      2      2
    53     20     14
     2    259    167
   190      2      9
    41     19    598
     2     15      5
    30     17     94
    12   6790    182
  5708   2729    479
   137    663      9
  2511      3    228
 12106      1      3
   691      1      1
  1164      1      1
    53      1      1
     3      1      1
[torch.cuda.LongTensor of size 38x3 (GPU 0)]

Variable containing:
    17    300     94
    51      6     12
    28      4     67
    18    186    400
  1654   5148      4
     9   